In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3,7"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
import torch
import torch.nn as nn
import copy
from transformers import TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from modelscope import snapshot_download
from modelscope import Qwen2_5OmniForConditionalGeneration, Qwen2_5OmniProcessor, Qwen2_5OmniThinkerForConditionalGeneration
import re
from transformers.models.qwen2_vl.video_processing_qwen2_vl import Qwen2VLVideoProcessor
from qwen_omni_utils import process_mm_info
import json
from torch.utils.data import Dataset
from transformers.image_utils import SizeDict
import torch._dynamo
torch._dynamo.config.suppress_errors = True

目前使用的是和longvale一样的多轮对话（）和单轮对话结合的数据加载

TODO
- 改成纯单轮数据
- 因为改成纯单轮数据，但是多个query实际结合相同的多模态特征传入，音视频 Token 应该缓存，只是文本 Token 和 label 更新

此外，要论证目前的固定帧采用对于 tokenization 的影响（各个模态 token 数及排列），证明调整是合理的

- 初始状态无修改的 tokenization；
- 初始状态 + 0.5fps + 上限 50fps；
- 初始状态 + 图像 resize 224；
- final；

In [2]:
class OmniStepMemoryTracker:
    def __init__(self, log_every=1):
        self.step = 0
        self.log_every = log_every

    def log(self, batch):
        if self.step % self.log_every != 0:
            self.step += 1
            return

        torch.cuda.synchronize()

        alloc = torch.cuda.memory_allocated() / 1024**2
        reserve = torch.cuda.memory_reserved() / 1024**2
        peak = torch.cuda.max_memory_allocated() / 1024**2

        # ---------- text ----------
        seq_len = batch["input_ids"].shape[1]
        label_tokens = (batch["labels"] != -100).sum().item()

        # ---------- video ----------
        if "video_grid_thw" in batch:
            t, h, w = batch["video_grid_thw"][0].tolist()
            video_tokens = t * h * w
        else:
            t = h = w = video_tokens = 0

        # ---------- audio ----------
        if "input_features" in batch:
            audio_tokens = batch["input_features"].shape[-1]
        else:
            audio_tokens = 0

        print(
            f"[Step {self.step:05d}] "
            f"seq={seq_len}, label={label_tokens} | "
            f"video={t}x{h}x{w}={video_tokens} | "
            f"audio={audio_tokens} | "
            f"CUDA alloc={alloc:.1f}MB peak={peak:.1f}MB reserve={reserve:.1f}MB"
        )

        self.step += 1

def replace_time_tokens_with_percentage(text, time_map, duration):

    if not time_map or duration is None:
        return text

    def repl(match):
        token = match.group(0)
        if token not in time_map:
            return token
        t = time_map[token]
        pct = t / duration * 100.0
        return f"{pct:.1f}%"

    return re.sub(r"<s\d+>|<e\d+>", repl, text)

class OmniVideoConversationDataset(Dataset):
    def __init__(self, json_path: str, video_root: str):
        with open(json_path, "r") as f:
            self.raw_data = json.load(f)

        self.video_root = video_root

    def __len__(self):
        return len(self.raw_data)

    def __getitem__(self, idx):
        item = self.raw_data[idx]

        video_id = item["id"]
        video_path = os.path.join(self.video_root, f"{video_id}.mp4")
        audio_path = video_path.replace(".mp4", ".wav")

        duration = item.get("meta", {}).get("duration", None)
        time_map = item.get("meta", {}).get("token", {})

        return {
            "video_path": video_path,
            "audio_path": audio_path,
            "conversations": copy.deepcopy(item["conversations"]),
            "duration": duration,
            "time_map": time_map,
        }

class QwenOmniDataCollator:
    def __init__(self, processor):
        self.processor = processor
        self.tokenizer = processor.tokenizer

    def _replace_time_tokens(self, conversations, time_map, duration):
        if not time_map or duration is None:
            return conversations

        def repl(match):
            token = match.group(0)
            if token not in time_map:
                return token
            pct = time_map[token] / duration * 100
            return f"{pct:.1f}%"

        for turn in conversations:
            turn["value"] = re.sub(r"<s\d+>|<e\d+>", repl, turn["value"])
        return conversations
    
    def _split_rounds(self, conversations):
        rounds = []
        cur = []
        for turn in conversations:
            cur.append(turn)
            if turn["from"] == "gpt":
                rounds.append(cur)
                cur = []
        return rounds

    def _truncate_by_round_with_labels(
        self,
        base_chat,          # system + video/audio
        rounds,             # [[h,g], [h,g], ...]
        max_total_tokens    # input + label 最大 token 数
    ):
        rounds = copy.deepcopy(rounds)

        while True:
            chat = copy.deepcopy(base_chat)

            # 统计 label token
            total_tokens = 0
            for r in rounds:
                for t in r:
                    role = "user" if t["from"] == "human" else "assistant"
                    chat.append({
                        "role": role,
                        "content": [{"type": "text", "text": t["value"]}],
                    })

            # 用 tokenizer 计算 input token 长度
            prompt = self.processor.apply_chat_template(
                chat, tokenize=False, add_generation_prompt=False
            )
            input_tokens = len(self.tokenizer(prompt).input_ids)

            # 统计 label token
            label_tokens = 0
            for r in rounds:
                for t in r:
                    if t["from"] == "gpt":  # 只计算 assistant 输出
                        label_tokens += len(self.tokenizer(t["value"]).input_ids)

            total_tokens = input_tokens + label_tokens
            print(input_tokens, label_tokens)

            # 如果总长度符合限制，或者只剩最后一轮，返回
            if total_tokens <= max_total_tokens:
                return chat

            # ❗ 删除最早的一整轮
            rounds = rounds[1:]


    def _build_conversation(self, sample):
        conversations = self._replace_time_tokens(
            sample["conversations"],
            sample["time_map"],
            sample["duration"],
        )

        base_chat = [
            {
                "role": "system",
                "content": [{"type": "text", "text": "You are Qwen, a virtual human developed by the Qwen Team, Alibaba Group, capable of perceiving auditory and visual inputs, as well as generating text and speech."}],
            },
            {
                "role": "user",
                "content": [
                    {"type": "video", "video": sample["video_path"], "fps": 0.5, "max_frames": 50},
                    # {"type": "video", "video": sample["video_path"]},
                    {"type": "audio", "audio": sample["audio_path"]},
                ],
            },
        ]

        rounds = self._split_rounds(conversations)

        chat = self._truncate_by_round_with_labels(
            base_chat=base_chat,
            rounds=rounds,
            max_total_tokens=2304
        )

        return chat

    def _build_labels(self, input_ids):
        labels = input_ids.clone()
        labels[:] = -100

        im_start = self.tokenizer.convert_tokens_to_ids("<|im_start|>")
        im_end = self.tokenizer.convert_tokens_to_ids("<|im_end|>")
        assistant = self.tokenizer.convert_tokens_to_ids("assistant")

        i = 0
        while i < len(input_ids) - 1:
            if input_ids[i] == im_start and input_ids[i + 1] == assistant:
                j = i + 3  # skip <|im_start|> assistant \n
                while j < len(input_ids) and input_ids[j] != im_end:
                    labels[j] = input_ids[j]
                    j += 1
                i = j
            else:
                i += 1
        return labels

    
    def __call__(self, features):
        texts, videos, audios = [], [], []

        for sample in features:
            conversation = self._build_conversation(sample)

            prompt = self.processor.apply_chat_template(
                conversation,
                tokenize=False,
                add_generation_prompt=False,
            )
            texts.append(prompt)

            audios_, _, videos_ = process_mm_info(
                    conversation, use_audio_in_video=False
                )

            videos.append(videos_[0] if videos_ else None)
            audios.append(audios_[0] if audios_ else None)

        batch = self.processor(
            text=texts,
            videos=videos,
            audio=audios,
            padding=True,
            return_tensors="pt",
            use_audio_in_video=False,
        )

        labels = torch.stack([
            self._build_labels(ids)
            for ids in batch["input_ids"]
        ])

        label_tokens = (labels != -100).sum().item()
        print("label tokens:", label_tokens)


        batch["labels"] = labels

        if not hasattr(self, "_debug_printed"):
            # self._debug_printed = True

            print("\n========== Omni Batch Debug ==========")

            # ---------- Text ----------
            print("[Text]")
            print("input_ids:", batch["input_ids"].shape)
            print("attention_mask:", batch["attention_mask"].shape)
            print("labels:", batch["labels"].shape)
            print(
                "label tokens:",
                (batch["labels"] != -100).sum().item()
            )

            # ---------- Video ----------
            if "pixel_values_videos" in batch:
                pv = batch["pixel_values_videos"]
                print("\n[Video]")
                print("pixel_values_videos:", pv.shape)
                print("dtype:", pv.dtype)
                print("video_grid_thw:", batch.get("video_grid_thw"))

                video_mem = pv.numel() * pv.element_size() / 1024**2
                print(f"video tensor size: {video_mem:.2f} MB")

            # ---------- Audio ----------
            for k in batch.keys():
                if "audio" in k or "input_features" in k:
                    v = batch[k]
                    if isinstance(v, torch.Tensor):
                        mem = v.numel() * v.element_size() / 1024**2
                        print("\n[Audio]")
                        print(f"{k}: {v.shape}, {mem:.2f} MB")

            print("=====================================\n")


        return batch



train_dataset = OmniVideoConversationDataset(
    json_path="../../LongVALE/data/longvale-sft-bp-7k.json",
    video_root="../../LongVALE/raw_videos_train/video_train_7240/"
)


model_path = snapshot_download(
    'Qwen/Qwen2.5-Omni-3B',
    cache_dir="../../Qwen/cache/modelscope"
)

model = Qwen2_5OmniThinkerForConditionalGeneration.from_pretrained(
    model_path,
    dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
    use_safetensors=True
)

class FixedResQwen2VLVideoProcessor(Qwen2VLVideoProcessor):
    def _preprocess(
        self, videos, do_resize=True, size=None, interpolation=None, **kwargs
    ):
        # 固定分辨率
        fixed_size = SizeDict(height=224, width=224)
        for i, video in enumerate(videos):
            videos[i] = self.resize(video, size=fixed_size, interpolation=interpolation)
        return super()._preprocess(videos, do_resize=False, size=fixed_size, interpolation=interpolation, **kwargs)
    
video_processor = FixedResQwen2VLVideoProcessor.from_pretrained(model_path)

processor = Qwen2_5OmniProcessor.from_pretrained(
    model_path,
    video_processor=video_processor,
)


# 配置LoRA
config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
)

model = get_peft_model(model, config)

for name, param in model.named_parameters():
    if (
        "audio_tower" in name
        or "visual" in name
    ):
        param.requires_grad = False

model.gradient_checkpointing_enable()
model.enable_input_require_grads()
model.config.use_cache = False

model.print_trainable_parameters()

# 检查模型是否在训练模式
model.train()
print(f"Model is in training mode: {model.training}")

batch_size = 1

args = TrainingArguments(
    output_dir="./r_models",
    remove_unused_columns=False,
    eval_strategy="no",
    save_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=1,
    bf16=True,
    fp16=False,
    fp16_full_eval=False,
    num_train_epochs=1,
    logging_steps=5,
    load_best_model_at_end=False,
)

data_collator = QwenOmniDataCollator(processor)

class DebugTrainer(Trainer):
    def __init__(self, *args, memory_tracker=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.memory_tracker = memory_tracker

    def training_step(self, model, inputs, *args, **kwargs):
        if self.memory_tracker is not None:
            self.memory_tracker.log(inputs)

        return super().training_step(model, inputs, *args, **kwargs)

memory_tracker = OmniStepMemoryTracker(log_every=1)


trainer = DebugTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    data_collator=data_collator,
    memory_tracker=memory_tracker,
)





2025-12-24 20:22:08,600 - modelscope - INFO - Target directory already exists, skipping creation.
Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.
Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


trainable params: 22,413,312 || all params: 4,734,622,720 || trainable%: 0.4734
Model is in training mode: True


In [ ]:
print("vocab size:", processor.tokenizer.vocab_size)
print("number of tokens:", len(processor.tokenizer))
print(processor.tokenizer.model_max_length)


vocab size: 151643
number of tokens: 151665
32768


In [ ]:
from inspect import signature
print(signature(Qwen2_5OmniForConditionalGeneration.forward))
print(signature(Qwen2_5OmniThinkerForConditionalGeneration.forward))

In [4]:
def verify_lora_gradients(model):
    """验证LoRA梯度流"""
    model.train()
    
    # 创建测试数据
    test_inputs = {
        'input_ids': torch.randint(0, 1000, (1, 10)).cuda(),
        'attention_mask': torch.ones(1, 10).cuda(),
        'labels': torch.randint(0, 1000, (1, 10)).cuda(),
    }
    
    # 前向传播
    outputs = model(**test_inputs)
    loss = outputs.loss
    
    print(f"Loss: {loss.item():.4f}")
    print(f"Loss requires_grad: {loss.requires_grad}")
    
    if loss.requires_grad:
        # 反向传播
        loss.backward()
        
        # 检查梯度
        gradients_found = False
        for name, param in model.named_parameters():
            if param.requires_grad and param.grad is not None:
                gradients_found = True
                grad_norm = param.grad.norm().item()
                if 'lora' in name:
                    print(f"  ✓ LoRA梯度: {name} | norm={grad_norm:.6f}")
        
        if not gradients_found:
            print("  ⚠ 没有找到梯度")
    else:
        print("  ✗ Loss没有requires_grad")

verify_lora_gradients(model)

Loss: 11.0864
Loss requires_grad: True
  ✓ LoRA梯度: base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight | norm=0.000000
  ✓ LoRA梯度: base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight | norm=0.191341
  ✓ LoRA梯度: base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight | norm=0.000000
  ✓ LoRA梯度: base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight | norm=0.245467
  ✓ LoRA梯度: base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight | norm=0.000000
  ✓ LoRA梯度: base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight | norm=2.072363
  ✓ LoRA梯度: base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight | norm=0.000000
  ✓ LoRA梯度: base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight | norm=1.310889
  ✓ LoRA梯度: base_model.model.model.layers.0.mlp.up_proj.lora_A.default.weight | norm=0.000000
  ✓ LoRA梯度: base_model.model.model.layers.0.mlp.up_proj.lora_B.default.weig

In [3]:
sample = train_dataset[0]

batch = data_collator([sample])

labels = batch["labels"]       # [batch_size, seq_len]
input_ids = batch["input_ids"] # [batch_size, seq_len]
tokenizer = processor.tokenizer

for i in range(labels.shape[0]):
    effective_ids = []
    ignored_ids = []
    
    for j in range(labels.shape[1]):
        token_id = input_ids[i, j].item()
        label = labels[i, j].item()
        if label != -100:
            effective_ids.append(token_id)
        else:
            ignored_ids.append(token_id)
    
    effective_text = tokenizer.decode(effective_ids)
    ignored_text = tokenizer.decode(ignored_ids)
    
    print(f"=== Sample {i} ===")
    print("有效 token 拼接文本:")
    print(effective_text)
    print("无效 token 拼接文本:")
    print(ignored_text)



543 125


qwen-vl-utils using decord to read video.
Token indices sequence length is longer than the specified maximum sequence length for this model (83507 > 32768). Running this sequence through the model will result in indexing errors


label tokens: 125

========== Omni Batch Debug ==========
[Text]
input_ids: torch.Size([1, 83507])
attention_mask: torch.Size([1, 83507])
labels: torch.Size([1, 83507])
label tokens: 125

[Video]
pixel_values_videos: torch.Size([306176, 1176])
dtype: torch.float32
video_grid_thw: tensor([[256,  26,  46]])
video tensor size: 1373.53 MB

[Audio]
input_features: torch.Size([1, 128, 30000]), 14.65 MB

=== Sample 0 ===
有效 token 拼接文本:
From 83.1% to 85.0%.From 0.0% to 66.5%.From 85.0% to 100.0%.From 73.2% to 80.3%.As the celebratory mood continues, the camera focuses on a plate laden with fried chicken, vegetables, and a lime wedge, panning out to show someone about to take a bite. Upbeat pop music floods the scene, a male vocalist singing about ignoring negativity and rumors.From 68.4% to 73.2%.
无效 token 拼接文本:
<|im_start|>system
You are Qwen, a virtual human developed by the Qwen Team, Alibaba Group, capable of perceiving auditory and visual inputs, as well as generating text and speech.<|im

In [ ]:
torch.cuda.synchronize()
print(
    f"Max memory allocated: {torch.cuda.max_memory_allocated() / 1024**2:.2f} MB"
)
print(
    f"Max memory reserved: {torch.cuda.max_memory_reserved() / 1024**2:.2f} MB"
)

=== Sample 0 ===
有效 token 拼接文本:
From <s4> to <e4>.

这里有个问题，<s4> 转化成什么时间
- 用真实时间？
    因为 Qwen 实际上能学习到真实的时间，但是评估的时候要算成相对时间吗？还是直接和数据集中的计算？但是这样和 LongVale 算是不是不太公平
- 和 LongVale 一样用相对时间百分比
    感觉这个是对应的固定 100 帧，如果 Qwen 也固定一百帧的话（不太确定这样能不能对应成绝对时间？），而且固定 100帧 目前显存也不够


In [ ]:
print("\n=== 进行前向传播测试 ===")
with torch.set_grad_enabled(True):

    # sample = next(iter(train_dataset))
    sample = train_dataset[13]
    batch = data_collator([sample])
    

    device = model.device
    batch = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in batch.items()}
    
    outputs = model(**batch)
    loss = outputs.loss
    
    print(f"Loss: {loss}")
    print(f"Loss requires_grad: {loss.requires_grad}")
    
    if loss.requires_grad:
        loss.backward()
        print("反向传播成功")
        

        for name, param in model.named_parameters():
            if param.requires_grad and param.grad is not None:
                print(f"参数 '{name}' 有梯度")
                break
        else:
            print("没有发现任何参数的梯度")
    else:
        print("loss没有requires_grad属性")

In [ ]:
print(len(train_dataset))

In [ ]:
trainer.get_train_dataloader()

In [ ]:
sample = train_dataset[0]
print(sample.keys())
print(sample["conversation"])

In [ ]:
collator = QwenOmniDataCollator(processor)
batch = collator([sample])
for k, v in batch.items():
    if isinstance(v, torch.Tensor):
        print(k, v.shape)

In [ ]:
train_loader = trainer.get_train_dataloader()
for b in train_loader:
    print(b.keys())
    break

In [ ]:
trainer.train()